In [1]:
#Install the required libraries
! pip install langchain_core
! pip install langchain
! pip install langchain_community
! pip install PyPDF2
! pip install langchain_huggingface
! pip install flashrank


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Import required libraries
import os
from typing import List
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from PyPDF2 import PdfReader
from langchain.retrievers.document_compressors import FlashrankRerank
from ScoredCrossEncoderReranker import ScoredCrossEncoderReranker  
from flashrank import Ranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
import time
from sklearn.metrics import ndcg_score

In [ ]:
def extract_texts_from_pdfs(folder_path):
    """Extract texts from all PDFs in the folder."""
    documents = []
    for idx, file_name in enumerate(os.listdir(folder_path)):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(folder_path, file_name)
            reader = PdfReader(file_path)
            pdf_text = ""
            for page in reader.pages:
                pdf_text += page.extract_text()
            
            # Create a Document object with metadata, including a unique ID
            document = Document(
                page_content=pdf_text.strip(),
                metadata={"id": idx + 1, "file_name": file_name}
            )
            documents.append(document)
    return documents

pdf_folder_path = "RAGMeUp/server/data" #Change path to your data folder

#Extract text from PDFs
documents = extract_texts_from_pdfs(pdf_folder_path)

#Check the extracted documents
for doc in documents:
    print(f"Document ID: {doc.metadata['id']}, File Name: {doc.metadata['file_name']} Done") 

# Get embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create a retriever
retriever = InMemoryVectorStore.from_documents(documents, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

Document ID: 1, File Name: 01 - Introduction.pdf Done
Document ID: 2, File Name: 02 - Linear Models.pdf Done
Document ID: 3, File Name: 03 - Kernelization.pdf Done
Document ID: 4, File Name: 04 - Model Selection.pdf Done
Document ID: 5, File Name: 05 - Ensemble Learning.pdf Done
Document ID: 6, File Name: 06 - Data Preprocessing.pdf Done
Document ID: 7, File Name: 07 - Bayesian Learning.pdf Done
Document ID: 8, File Name: 08 - Neural Networks.pdf Done
Document ID: 9, File Name: 09 - Convolutional Neural Networks.pdf Done
Document ID: 10, File Name: 10 - Neural Networks for text.pdf Done
Document ID: 11, File Name: 1607.04606.pdf Done
Document ID: 12, File Name: 1706.03762.pdf Done
Document ID: 13, File Name: 2021-SSCI-AComparativeStudyOfFuzzyTopicModelsAndLDAInTermsOfInterpretability.pdf Done
Document ID: 14, File Name: 2022-FrontiersinBigData.pdf Done
Document ID: 15, File Name: 2022.acl-long.225.pdf Done
Document ID: 16, File Name: 2022.emnlp-main.759.pdf Done
Document ID: 17, File N

c:\Users\20184113\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Importing FlasrankRerank model and tracking time how long it takes to run the model and get the relevant documents

start_time_flashrank = time.time()
model_name = "ms-marco-MultiBERT-L-12"  
flashrank_client = Ranker(model_name=model_name)

compressor_flashrank = FlashrankRerank(client=flashrank_client, top_n=5, model=model_name)

compression_retriever_flashrank = ContextualCompressionRetriever(
    base_compressor=compressor_flashrank, base_retriever=retriever
)

compressed_docs_flashrank = compression_retriever_flashrank.invoke("What is Deep Learning?")
time_passed_flashrank =  time.time() - start_time_flashrank

In [5]:
#Importing ScoredCrossEncoderReranker with Reodering model and tracking time how long it takes to run the model and get the relevant documents

start_time_SCER = time.time()
scored_reranker = ScoredCrossEncoderReranker(
    model=HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"),
    top_n=5)

compression_retriever_scored = ContextualCompressionRetriever(
    base_compressor=scored_reranker, base_retriever=retriever
)

compressed_doc_scored = compression_retriever_scored.invoke("What is Deep Learning?")

time_passed_SCER = time.time() - start_time_SCER

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cpu
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


In [6]:
#The difference in running time needed for the two models
print("Time passed to run ScoredCrossEncoder:", time_passed_flashrank)
print("Time passed to run ScoredCrossEncoder:", time_passed_SCER)

Time passed to run ScoredCrossEncoder: 7.132648468017578
Time passed to run ScoredCrossEncoder: 2.509944200515747


In [7]:
#Show a comparison between the two reranker models in terms of relevance scores for the documents
print("\nComparison of Results:")
for idx, (flash_doc, scored_doc) in enumerate(zip(compressed_docs_flashrank, compressed_doc_scored), 1):
    #print(f"Document {idx}:")
    print(f"Flashrank:  Metadata: {flash_doc.metadata}")
    print(f"ScoredCrossEncoder:Metadata: {scored_doc.metadata}")
    print("-" * 50)


Comparison of Results:
Flashrank:  Metadata: {'id': 30, 'relevance_score': np.float32(0.9984581), 'file_name': 'deep_learning_1.pdf'}
ScoredCrossEncoder:Metadata: {'id': 30, 'file_name': 'deep_learning_1.pdf', 'relevance_score': np.float32(1.0)}
--------------------------------------------------
Flashrank:  Metadata: {'id': 31, 'relevance_score': np.float32(0.9974198), 'file_name': 'deep_learning_2.pdf'}
ScoredCrossEncoder:Metadata: {'id': 9, 'file_name': '09 - Convolutional Neural Networks.pdf', 'relevance_score': np.float32(0.0)}
--------------------------------------------------
Flashrank:  Metadata: {'id': 28, 'relevance_score': np.float32(0.9952842), 'file_name': '978-981-10-5209-5_1.pdf'}
ScoredCrossEncoder:Metadata: {'id': 28, 'file_name': '978-981-10-5209-5_1.pdf', 'relevance_score': np.float32(0.9557583)}
--------------------------------------------------
Flashrank:  Metadata: {'id': 4, 'relevance_score': np.float32(0.99440783), 'file_name': '04 - Model Selection.pdf'}
Scored

In [8]:
#Calculating the NDCG score to see how the model is working overall in terms of relevance score
true_relevance = [5, 4, 3, 2, 1]  
original_relevance = [doc.metadata["relevance_score"] for doc in compressed_docs_flashrank]
modified_relevance = [doc.metadata["relevance_score"] for doc in compressed_doc_scored]

# Compute NDCG
original_ndcg = ndcg_score([true_relevance], [original_relevance])
modified_ndcg = ndcg_score([true_relevance], [modified_relevance])

print(f"FlashrankReranker NDCG: {original_ndcg}")
print(f"ScoredCrossEncoderReranker with Reordering: {modified_ndcg}")

FlashrankReranker NDCG: 1.0
ScoredCrossEncoderReranker with Reordering: 0.9542080637541479
